In [1]:
# pip install pyodbc

In [2]:
import pandas as pd
import pyodbc
import os

In [3]:
# Define the parameters for connect to SQL Server database
driver_name = "ODBC Driver 17 for SQL Server"
server_name = "TIT-20230930FXD\\SQLEXPRESS"
database_name = "brazilian_ecommerce" # using database_name = "master" for create new database
trusted_connection = "yes"

# Define connection string
conn_str = f"DRIVER={driver_name};SERVER={server_name};DATABASE={database_name};Trusted_Connection={trusted_connection}"

In [4]:
# Read CSV file
data_path = r"C:\Users\Admin\Documents\GitHub\data-science-bootcamp9\09_Essential Python for Data Analyst\Project - Create and add table to SQL Server Database\data"
file_name = "olist_products_dataset.csv"
df = pd.read_csv(os.path.join(data_path, file_name), encoding="utf8")
df.dtypes

product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_photos_qty            float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

In [5]:
df.duplicated().sum()

0

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [8]:
df = df.astype(object)
df.dtypes

product_id                    object
product_category_name         object
product_name_lenght           object
product_description_lenght    object
product_photos_qty            object
product_weight_g              object
product_length_cm             object
product_height_cm             object
product_width_cm              object
dtype: object

In [9]:
# Convert NaN values to None
df = df.where(pd.notnull(df), None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   product_id                  32951 non-null  object
 1   product_category_name       32341 non-null  object
 2   product_name_lenght         32341 non-null  object
 3   product_description_lenght  32341 non-null  object
 4   product_photos_qty          32341 non-null  object
 5   product_weight_g            32949 non-null  object
 6   product_length_cm           32949 non-null  object
 7   product_height_cm           32949 non-null  object
 8   product_width_cm            32949 non-null  object
dtypes: object(9)
memory usage: 2.3+ MB


In [ ]:
df.nunique()

In [10]:
print(df.product_id.astype(str).str.len().max())
print(df.product_category_name.astype(str).str.len().max())

32
46


In [11]:
# drop_table_query = """
#     DROP TABLE IF EXISTS products
# """

# create_table_query = """
#     CREATE TABLE products(    
#         product_id VARCHAR(50) NOT NULL PRIMARY KEY,
#         product_category_name VARCHAR(50) NULL,
#         product_name_lenght FLOAT NULL,
#         product_description_lenght FLOAT NULL,
#         product_photos_qty FLOAT NULL,
#         product_weight_g FLOAT NULL,
#         product_length_cm FLOAT NULL,
#         product_height_cm FLOAT NULL,
#         product_width_cm FLOAT NULL
#     );
# """

# with pyodbc.connect(conn_str) as con:
#     con.execute(drop_table_query)
#     con.execute(create_table_query)
#     con.commit()

In [12]:
# Define SQL command for truncate table
truncate_query = """
    TRUNCATE TABLE products
"""

# Execute command SQL for truncate table
with pyodbc.connect(conn_str) as con:
    cursor = con.cursor()
    cursor.execute(truncate_query)
    con.commit()

In [13]:
with pyodbc.connect(conn_str) as con:
    cursor = con.cursor()
    for index, row in df.iterrows():
        insert_query = """
            INSERT INTO products
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        # Execute command SQL for insert new data
        cursor.execute(insert_query, tuple(row))
    con.commit()

In [14]:
with pyodbc.connect(conn_str) as con:
    df_query = pd.read_sql("SELECT * FROM products", con)
    
df_query.info()

C:\Users\Admin\AppData\Local\Temp\ipykernel_13284\2263898300.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql("SELECT * FROM products", con)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB
